In [1]:
import numpy as np
from numba import cuda
import math
import cupy as cp
import cudf
import yfinance as yf

/home/josgt/anaconda3/envs/tradingapp/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
def dropLabels(events,minPct=.05):
    # apply weights, drop labels with insufficient examples
    while True:
        # Count the number of observations the prediction feature has for each label
        df0=events['y'].value_counts(normalize=True)
        # If the label with minimum number of observations is lower than the minPct threshold 
        # or the number of labels in the prediction features is 2, then break the while loop
        if (df0.min()>minPct) or (df0.shape[0]<3):break
        # Drop the label with minimum number of observations
        events = events[events['y']!=df0.index[df0.argmin()]]
    return events

def get_Daily_Volatility(df,span0=20):
    # simple percentage returns
    df0=df['Close'].pct_change()
    # 20 days, a month EWM's std as boundary
    df0=df0.ewm(span = span0, adjust = False).std()
    # Round the column values to six decimals
    df0 = df0.round(6)
    return df0

In [3]:
@cuda.jit
def triple_barrier_method_cuda(close, high, low, daily_volatility, upper_lower_multipliers, holding_period, out):

    # Set the number days passed to zero
    days_passed = 0
    # Set the vertical barrier initial value to NaN
    vert_barrier = math.nan
    # Set the top barrier initial value to NaN
    top_barrier = math.nan
    # Set the bottom barrier initial value to NaN
    bottom_barrier = math.nan
    # Set the absolute position of the used 1-dimension thread index
    i = cuda.grid(1)

    # Whenever we don't get to the last observation of the Close price column
    if i < close.shape[0]:
        # Set the days passed as the thread index number plus 1
        days_passed = float(i+1)

        # If the next hold period is located before the last column observation and the holding period is different from zero
        if ((days_passed + holding_period) < close.shape[0]) and (holding_period != 0):
            # Set the vertical barrier as the holding period
            vert_barrier = days_passed + float(holding_period)

        # If the upper multiplier is higher than 0
        if upper_lower_multipliers[0] > 0:
            # Set the top barrier
            top_barrier = round(close[i] + close[i] * upper_lower_multipliers[0] * daily_volatility[i],6)

        # If the lower multiplier is higher than 0
        if upper_lower_multipliers[1] > 0:
            # Set the bottom multiplier
            bottom_barrier = round(close[i] - close[i] * upper_lower_multipliers[1] * daily_volatility[i],6)

        # Set the days passed value for the function output at day i
        out[i, 0] = days_passed
        # Set the Close price value for the function output at day i
        out[i, 1] = close[i]
        # Set the vertical barrier value for the function output at day i
        out[i, 2] = float(vert_barrier)
        # Set the top barrier value for the function output at day i
        out[i, 3] = top_barrier
        # Set the bottom barrier value for the function output at day i
        out[i, 4] = bottom_barrier

        cuda.syncthreads()

        # Set the start day from the holding period
        start = int(out[i, 0])
        # Set the end day from the holding period
        end = int(out[i, 2])

        # If the holding period is not zero and the row number is lower than the last holding-period days of the data sample
        if (holding_period != 0) and ((days_passed + holding_period) < close.shape[0]):
            # Set the top barrier
            top_barrier = out[i, 3]
            # Set the bottom barrier
            bottom_barrier = out[i, 4]

            # Set the condition that tells if the take profit target was breached
            condition_close_pt = False
            # Set the condition that tells if the stop loss target was breached
            condition_close_sl = False
            # Set the condition that tells if the top barrier hits the high price to False
            condition_high = False
            # Set the condition that tells if the bottom barrier hits the low price to False
            condition_low = False

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the top barrier is breached by the close price
                if round(close[j],6) >= top_barrier:
                    # Set the take profit condition to True and break the loop
                    condition_close_pt = True
                    break

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the bottom barrier is breached by the close price
                if round(close[j],6) <= bottom_barrier:
                    condition_close_sl = True
                    break

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the top barrier is breached by the high price
                if high[j] >= top_barrier:
                    condition_high = True
                    break

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the bottom barrier is breached by the low price
                if low[j] <= bottom_barrier:
                    condition_low = True
                    break

            # If the take profit target or high condition is True 
            if condition_close_pt or condition_high:
                # Set the prediction feature to +1
                out[i, 5] = 1.0
            # If the take profit target or high condition is True
            elif condition_close_sl or condition_low:
                # Set the prediction feature to -1
                out[i, 5] = -1.0
            # If any condition is breached
            else:
                # If the holding-period-to-current close price return is higher than zero
                if (close[end]/close[i] - 1) > 0.0:
                    # Set the prediction feature to +1
                    out[i, 5] = 1.0
                # If the holding-period-to-current close price return is lower than zero
                elif (close[end]/close[i] - 1) < 0.0:
                    # Set the prediction feature to -1
                    out[i, 5] = -1.0
                # If the holding-period-to-current close price return is equal to zero    
                else:
                    # Set the prediction feature to zero
                    out[i, 5] = 0.0
        # If the current day is 
        else:
            out[i, 5] = 0.0

In [4]:
def triple_barrier_method(df, holding_period=10, upper_lower_multipliers=[2, 1]):
    # Set the close price as a cupy array
    close = cp.array(df['Close'].values, dtype=cp.float64)
    # Set the high price as a cupy array
    high = cp.array(df['High'].values, dtype=cp.float64)
    # Set the low price as a cupy array
    low = cp.array(df['Low'].values, dtype=cp.float64)
    # Set the daily volatility as a cupy array
    daily_volatility = cp.array(df['vol'].values, dtype=cp.float64)
    # Set the barriers empty array as a cupy array
    barriers = np.empty((close.shape[0], 6), dtype=cp.float64)
    # Set the upper and lower multipliers as a cupy array
    upper_lower_multipliers = np.array(upper_lower_multipliers, dtype=cp.float64)

    # Set the block size
    block_size = 256
    # Set the number of blocks
    num_blocks = (close.shape[0] - 1) // block_size + 1

    # Compute the barriers output
    triple_barrier_method_cuda[num_blocks, block_size](close, high, low, daily_volatility, upper_lower_multipliers, holding_period, barriers)

    # Transform the barriers array into a cudf dataframe
    barriers_df = cudf.DataFrame(barriers, columns=['days_passed', 'price', 'vert_barrier', 'top_barrier', 'bottom_barrier', 'out'])

    # Set the barrieres index to 
    barriers_df.set_index(df.index[-len(barriers_df.index):], inplace=True)

    # Transform the dataframe from cudf-based to pandas
    barriers_df = barriers_df.to_pandas()

    # Return the prediction feature
    return barriers_df[['out']]

In [5]:
# Import the Apple dataframe
data = yf.download('AAPL', start='1990-01-01', end='2024-04-04', auto_adjust=True)

# Prin the dataframe
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
1990-01-02,0.248953,0.264843,0.247187,0.263078,183198400
1990-01-03,0.268375,0.268375,0.264843,0.264843,207995200
1990-01-04,0.270141,0.273672,0.263078,0.265727,221513600
1990-01-05,0.266610,0.270141,0.261312,0.266610,123312000
1990-01-08,0.264843,0.268375,0.261312,0.268375,101572800
...,...,...,...,...,...
2024-03-27,170.410004,173.600006,170.110001,173.309998,60273300
2024-03-28,171.750000,172.229996,170.509995,171.479996,65672700
2024-04-01,171.190002,171.250000,169.479996,170.029999,46240500


In [6]:
# Compute the daily volatility
data['vol'] = get_Daily_Volatility(data)

# Drop the rows that have NaN values
data.dropna(inplace=True)

In [7]:
# Obtain the prediction feature with the above function
data['y'] = triple_barrier_method(data, holding_period=10, upper_lower_multipliers=[2, 1])

/home/josgt/anaconda3/envs/tradingapp/lib/python3.10/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 34 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/josgt/anaconda3/envs/tradingapp/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [8]:
data['y']

Date
1990-01-04    1.0
1990-01-05    1.0
1990-01-08   -1.0
1990-01-09   -1.0
1990-01-10   -1.0
             ... 
2024-03-27    0.0
2024-03-28    0.0
2024-04-01    0.0
2024-04-02    0.0
2024-04-03    0.0
Name: y, Length: 8627, dtype: float64

In [9]:
data['y'].value_counts()

 1.0    4880
-1.0    3735
 0.0      12
Name: y, dtype: int64

In [ ]:
import numpy as np
from numba import cuda
import math
import cupy as cp
import cudf
import yfinance as yf

def dropLabels(events,minPct=.05):
    # apply weights, drop labels with insufficient examples
    while True:
        # Count the number of observations the prediction feature has for each label
        df0=events['y'].value_counts(normalize=True)
        # If the label with minimum number of observations is lower than the minPct threshold 
        # or the number of labels in the prediction features is 2, then break the while loop
        if (df0.min()>minPct) or (df0.shape[0]<3):break
        # Drop the label with minimum number of observations
        events = events[events['y']!=df0.index[df0.argmin()]]
    return events

def get_Daily_Volatility(df,span0=20):
    # simple percentage returns
    df0=df['Close'].pct_change()
    # 20 days, a month EWM's std as boundary
    df0=df0.ewm(span = span0, adjust = False).std()
    # Round the column values to six decimals
    df0 = df0.round(6)
    return df0

@cuda.jit
def triple_barrier_method_cuda(close, high, low, daily_volatility, upper_lower_multipliers, holding_period, out):

    # Set the number days passed to zero
    days_passed = 0
    # Set the vertical barrier initial value to NaN
    vert_barrier = math.nan
    # Set the top barrier initial value to NaN
    top_barrier = math.nan
    # Set the bottom barrier initial value to NaN
    bottom_barrier = math.nan
    # Set the absolute position of the used 1-dimension thread index
    i = cuda.grid(1)

    # Whenever we don't get to the last observation of the Close price column
    if i < close.shape[0]:
        # Set the days passed as the thread index number plus 1
        days_passed = float(i+1)

        # If the next hold period is located before the last column observation and the holding period is different from zero
        if ((days_passed + holding_period) < close.shape[0]) and (holding_period != 0):
            # Set the vertical barrier as the holding period
            vert_barrier = days_passed + float(holding_period)

        # If the upper multiplier is higher than 0
        if upper_lower_multipliers[0] > 0:
            # Set the top barrier
            top_barrier = round(close[i] + close[i] * upper_lower_multipliers[0] * daily_volatility[i],6)

        # If the lower multiplier is higher than 0
        if upper_lower_multipliers[1] > 0:
            # Set the bottom multiplier
            bottom_barrier = round(close[i] - close[i] * upper_lower_multipliers[1] * daily_volatility[i],6)

        # Set the days passed value for the function output at day i
        out[i, 0] = days_passed
        # Set the Close price value for the function output at day i
        out[i, 1] = close[i]
        # Set the vertical barrier value for the function output at day i
        out[i, 2] = float(vert_barrier)
        # Set the top barrier value for the function output at day i
        out[i, 3] = top_barrier
        # Set the bottom barrier value for the function output at day i
        out[i, 4] = bottom_barrier

        cuda.syncthreads()

        # Set the start day from the holding period
        start = int(out[i, 0])
        # Set the end day from the holding period
        end = int(out[i, 2])

        # If the holding period is not zero and the row number is lower than the last holding-period days of the data sample
        if (holding_period != 0) and ((days_passed + holding_period) < close.shape[0]):
            # Set the top barrier
            top_barrier = out[i, 3]
            # Set the bottom barrier
            bottom_barrier = out[i, 4]

            # Set the condition that tells if the take profit target was breached
            condition_close_pt = False
            # Set the condition that tells if the stop loss target was breached
            condition_close_sl = False
            # Set the condition that tells if the top barrier hits the high price to False
            condition_high = False
            # Set the condition that tells if the bottom barrier hits the low price to False
            condition_low = False

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the top barrier is breached by the close price
                if round(close[j],6) >= top_barrier:
                    # Set the take profit condition to True and break the loop
                    condition_close_pt = True
                    break

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the bottom barrier is breached by the close price
                if round(close[j],6) <= bottom_barrier:
                    condition_close_sl = True
                    break

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the top barrier is breached by the high price
                if high[j] >= top_barrier:
                    condition_high = True
                    break

            # Set a for loop to check the barrier breaching for the next days until the holding period
            for j in range(start, end):
                # If the bottom barrier is breached by the low price
                if low[j] <= bottom_barrier:
                    condition_low = True
                    break

            # If the take profit target or high condition is True 
            if condition_close_pt or condition_high:
                # Set the prediction feature to +1
                out[i, 5] = 1.0
            # If the take profit target or high condition is True
            elif condition_close_sl or condition_low:
                # Set the prediction feature to -1
                out[i, 5] = -1.0
            # If any condition is breached
            else:
                # If the holding-period-to-current close price return is higher than zero
                if (close[end]/close[i] - 1) > 0.0:
                    # Set the prediction feature to +1
                    out[i, 5] = 1.0
                # If the holding-period-to-current close price return is lower than zero
                elif (close[end]/close[i] - 1) < 0.0:
                    # Set the prediction feature to -1
                    out[i, 5] = -1.0
                # If the holding-period-to-current close price return is equal to zero    
                else:
                    # Set the prediction feature to zero
                    out[i, 5] = 0.0
        # If the current day is 
        else:
            out[i, 5] = 0.0

def triple_barrier_method(df, holding_period=10, upper_lower_multipliers=[2, 1]):
    # Set the close price as a cupy array
    close = cp.array(df['Close'].values, dtype=cp.float64)
    # Set the high price as a cupy array
    high = cp.array(df['High'].values, dtype=cp.float64)
    # Set the low price as a cupy array
    low = cp.array(df['Low'].values, dtype=cp.float64)
    # Set the daily volatility as a cupy array
    daily_volatility = cp.array(df['vol'].values, dtype=cp.float64)
    # Set the barriers empty array as a cupy array
    barriers = np.empty((close.shape[0], 6), dtype=cp.float64)
    # Set the upper and lower multipliers as a cupy array
    upper_lower_multipliers = np.array(upper_lower_multipliers, dtype=cp.float64)

    # Set the block size
    block_size = 256
    # Set the number of blocks
    num_blocks = (close.shape[0] - 1) // block_size + 1

    # Compute the barriers output
    triple_barrier_method_cuda[num_blocks, block_size](close, high, low, daily_volatility, upper_lower_multipliers, holding_period, barriers)

    # Transform the barriers array into a cudf dataframe
    barriers_df = cudf.DataFrame(barriers, columns=['days_passed', 'price', 'vert_barrier', 'top_barrier', 'bottom_barrier', 'out'])

    # Set the barrieres index to 
    barriers_df.set_index(df.index[-len(barriers_df.index):], inplace=True)

    # Transform the dataframe from cudf-based to pandas
    barriers_df = barriers_df.to_pandas()

    # Return the prediction feature
    return barriers_df[['out']]

# Import the Apple dataframe
data = yf.download('AAPL', start='1990-01-01', end='2024-04-04', auto_adjust=True)

# Print the dataframe
data

# Compute the daily volatility
data['vol'] = get_Daily_Volatility(data)

# Drop the rows that have NaN values
data.dropna(inplace=True)

# Obtain the prediction feature with the above function
data['y'] = triple_barrier_method(data, holding_period=10, upper_lower_multipliers=[2, 1])

data['y'].value_counts()